# Watson Conversation Model Tester
This notebook demonstrates a simple framework for training and testing IBM Watson Conversation API 
By managing the training process programmatically, the intent recognition performance can be reliably tested with a truly blind test set.

In [1]:
# install watson-developer-cloud library
!pip install --upgrade watson-developer-cloud

# install sklearn (aka SciKit-Learn)
!pip install sklearn --upgrade

# import prerequisite libraries
import pandas as pd
import numpy as np
from bokeh.charts import Histogram, output_file, show
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


Requirement already up-to-date: watson-developer-cloud in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages
Requirement already up-to-date: pysolr<4.0,>=3.3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: requests<3.0,>=2.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: pyOpenSSL>=16.2.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from requests<3.0,>=2.0->watson-developer-cloud)
Requirement alre

In [4]:
# The code was removed by DSX for sharing.

Loading training and test data
Data loaded. Number of records: example    879
intent     879
dtype: int64


NameError: name 'Print' is not defined

In [13]:
# store into variable df
try:
    df = df_data_1
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    print('Follow the instructions to insert the pandas DataFrame above, and edit to')
    print('make the generated df_data_# variable match the variable used here.')
    raise
# split data into training and test sets
# typically training data is 80%. However, we should investigate this and confirm leading practices
train, test = train_test_split(df, test_size = 0.35)
# supply workspace and authentication parameters
CONVERSATION_USERNAME = '88b92c88-94a1-4226-ae57-9ca74bb12e57'
CONVERSATION_PASSWORD = '2CMxBdqB0TOo'
VERSION = '2017-05-26'
WORKSPACE_ID = '163bf2ef-4a65-4bb5-ae87-14df6870b573'
# create the authtication json object
import json
from watson_developer_cloud import ConversationV1
conversation = ConversationV1(
    username=CONVERSATION_USERNAME,
    password=CONVERSATION_PASSWORD,
    version= VERSION
)
#Clear the workspace of all existing intents
intents = conversation.list_intents(workspace_id=WORKSPACE_ID)['intents']
for intent in intents:
    conversation.delete_intent(workspace_id=WORKSPACE_ID, intent=intent['intent'])
print("Populating the model with intents")
# now create intent for each intent in the the training data
for intent in set([x for x in train['intent']]):
    conversation.create_intent(workspace_id=WORKSPACE_ID, intent=intent, description=intent)
# now add examples
print("Populating the model with examples")
for training_data in [x[1] for x in train[:].iterrows()]:
    conversation.create_example(workspace_id=WORKSPACE_ID, intent=training_data.intent, 
                                text=training_data.example)
print("Testing model with test data")
# now test with the test data
results = []
for test_data in [x[1] for x in test[:].iterrows()]:
    try:
        results.append(1 if conversation.message(workspace_id=WORKSPACE_ID, message_input=
            {"text": test_data.example})['intents'][0]['intent'] == test_data.intent else 0)
    except:
        results.append(0)
results = np.array(results)

print("Intent Recognizer Performance: {:.2%}".format(np.sum(results) / results.size))


Populating the model with intents
Populating the model with examples
Testing model with test data
Intent Recognizer Performance: 75.32%
